## TÍNH TOÁN VÀ VẼ BIỂU ĐỒ TRÒN CỦA TRANG BỊ TỘC VÀ HỆ

In [4]:
import pandas as pd
import numpy as np
import ast
from itertools import chain

### ĐỌC DỮ LIỆU ĐỂ LẤY TẤT CẢ CÁC MÓN ĐỒ

In [5]:
'''
Lọc trùng theo mã trận đấu và danh sách đồ vì:
* Các người chơi sẽ gặp nhau ở các trận đấu nên mã trận đấu sẽ bị trùng giữa các thư mục ngườichơi
* Có 127 trang bị chia thành 4 loại, mỗi champion có tối đa 3 trang bị chưa kể 1 trận có từ 8-10 trang bị và 6-9 champion 
mà mỗi champion có thể để từ 0-3 trang bị nên số trường hợp tối đa là rất lớn mà dữ liệu đầu vào chỉ có 32000 đội hình nên các bộ trang bị
sẽ không thể bị trùng giữa 2 người chơi khác nhau được!
'''
item_df = pd.read_csv('../Data/process_data_tft_each_item_csv.csv', usecols=[
                      'ID Match', 'List Name Item']).drop_duplicates().reset_index(drop=True)
# item_df = pd.read_csv('process_data_tft_each_item_csv.csv').drop_duplicates()

số lượng trận đấu thực tế từ dữ liệu rút trích về

In [6]:
len(item_df)

18096

### LẤY RA SỐ LƯỢNG ĐỒ CÓ TRONG TRẬN ĐẤU
vì dữ liệu list được lưu vào csv thì sẽ tự động biến đổi thành 1 string. vì thế ta phải tự mình biến đổi về lại list. May mắn thay đã có thư viện làm việc đó giúp ta :)

**ast.literak_eval**: dùng để biến đổi string thành list

In [7]:
list_items = []
for i in range(len(item_df['List Name Item'])):
    list_items.append(ast.literal_eval(item_df['List Name Item'][i])) 

### TÌM RA MÓN ĐỒ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

ta giảm chiều list item lại bằng list(chain.from_iterable)

hàm zip dùng để biến 2 list được cho ra thành key và values

list(chain) dùng để ép mảng xuống còn 1D và sau đó sort lại

sau đó ta rút trích ra được những món đồ tốt nhất và món đồ tệ nhất

In [8]:
def get_best_worst_items(list_items):
    flatten_list = list(chain.from_iterable(list_items))
    flatten_list = list(chain.from_iterable(flatten_list))
    flatten_list = np.array(flatten_list, dtype = object)
    unique, counts = np.unique(flatten_list, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_item = list(b.items())
    worst_item = list(b.items())[0:5]
    return best_item,worst_item

In [9]:
best_item,worst_item = get_best_worst_items(list_items)

5 món đồ tốt nhất và tệ nhất

In [10]:
print(best_item[:5])

[('TFT5_Item_MorellonomiconRadiant', 12), ('TFT5_Item_ShroudOfStillnessRadiant', 12), ('TFT5_Item_FrozenHeartRadiant', 14), ('TFT5_Item_LastWhisperRadiant', 14), ('TFT5_Item_RedemptionRadiant', 14)]


In [11]:
print(worst_item)

[('TFT5_Item_MorellonomiconRadiant', 12), ('TFT5_Item_ShroudOfStillnessRadiant', 12), ('TFT5_Item_FrozenHeartRadiant', 14), ('TFT5_Item_LastWhisperRadiant', 14), ('TFT5_Item_RedemptionRadiant', 14)]


### TÌM VỊ TRÍ TRUNG BÌNH CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU


khi người chơi sử dụng món đồ đó thì sẽ được lọt vào top mấy của game

In [15]:
average_df = pd.read_csv('../Data/process_data_tft_each_item_csv.csv', usecols = ['Puuid', 'Placement', 'List Name Item']).drop_duplicates().reset_index(drop = True)

chuyển đổi string sang list nhưng lần này ta để trong hàm để dễ sử dụng và nhìn trực quan hơn

In [16]:
def convert_string_to_list(ok, placement, column):
    temp = ok[ok.eq(placement).any(1)].reset_index(drop=True)
    top = []
    for i in range(len(temp)):
        top.append(ast.literal_eval(temp[column][i]))
    return top

lọc ra trong những top từ 1 đến 8 số lần item đó được sử dụng. Vòng for để duyệt từng top, sau đó lấy ra số lần sử dụn item đó

In [17]:
item_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(average_df, i, 'List Name Item')
    temp1 = 'top_' + str(i)
    item_on_top[temp1] = get_best_worst_items(temp)

5 item tốt nhất mà người chơi top 1 sử dụng

nếu muốn xem các top khác thì chỉ cần thay đổi string"top_1" thành các top khác

In [19]:
best_item_top_1, worst_item_top_1 = item_on_top['top_1']
print(best_item_top_1[-5:])

[('TFT_Item_SeraphsEmbrace', 1054), ('TFT_Item_IonicSpark', 1114), ('TFT_Item_JeweledGauntlet', 1184), ('TFT_Item_RedBuff', 1194), ('TFT_Item_MadredsBloodrazor', 1510)]


và 5 item tệ nhất mà người chơi top 1 sử dụng

In [22]:
print(worst_item_top_1)

[('TFT5_Item_RapidFirecannonRadiant', 1), ('TFT5_Item_StatikkShivRadiant', 1), ('TFT5_Item_ArchangelsStaffRadiant', 2), ('TFT5_Item_ChaliceOfPowerRadiant', 2), ('TFT5_Item_GargoyleStoneplateRadiant', 2)]


### TỈ LỆ CỦA MÓN ĐỒ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA MÓN ĐỒ TRONG TỪNG TOP CỦA TRẬN ĐẤU

In [23]:
all_item_df = pd.read_csv("../Data/item_tft_csv.csv", usecols=["Item"])

In [24]:
all_items = all_item_df['Item'].to_list()

item_avg_top: vị trí trung bình của món đồ khi ở trong top từ 1 đến 8

Nghĩa là khi món đồ đó được sử dụng thì trung bình bạn sẽ vào được top mấy

item_avg_on_match: tỉ lệ số trận đấu từ top 1 đến 8 mà món đồ đó được sử dụng

Nghĩa là tần suất sử dụng của món đồ đó trong từng top

In [26]:
sum = 0
temp = 0
item_avg_top = []
item_avg_on_match = []
match_each_top = []
for item in all_items:
    temp1 = []
    temp2 = []
    for i in range(1,9,1):
        cc = dict(item_on_top['top_' + str(i)][0])
        if(item in cc):
            temp = temp+dict(item_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(item_on_top['top_' + str(i)][0])[item])
        temp2.append(sum/32000)
        item_avg_on_match.append(temp2)
    temp1.append(temp/sum)
    item_avg_top.append(temp1)
    temp = 0
    sum = 0

như ta thấy ở đây, món đồ đầu tiên sẽ có avg_top là 4.03: nghĩa là nếu bạn sử dụng món đồ đó thì tỉ lệ top mà bạn đạt được là 4.03. còn avg_on_match: nghĩa là món đồ đó trong từng top thì tần suất xuất hiện là bao nhiêu

In [27]:
print(item_avg_top[0], item_avg_on_match[0])

[4.0321428571428575] [0.00153125, 0.00290625, 0.00409375, 0.00509375, 0.0063125, 0.007, 0.00778125, 0.00875]


## TƯƠNG TỰ VỚI TỘC VÀ HỆ

#### đọc dữ liệu để lấy hệ và tộc

In [31]:
traits_df = pd.read_csv('../Data/process_data_tft_general_csv.csv', usecols = ['ID Match','List Traits','List Number Items']).drop_duplicates().reset_index(drop=True)

In [32]:
list_traits = []

for i in range(len(traits_df['List Traits'])):
    list_traits.append(ast.literal_eval(traits_df['List Traits'][i])) 

### TÌM RA TỘC HỆ ĐƯỢC SỬ DỤNG NHIỀU NHẤT VÀ ÍT NHẤT

In [33]:
def get_best_worst_traits(list_traits):
    flatten_list = list(chain.from_iterable(list_traits))
    flatten_list = np.array(flatten_list, dtype = object)
    cc = flatten_list[0::3]
    unique, counts= np.unique(cc, return_counts=True)
    a = dict(zip(unique, counts))
    b = dict(sorted(a.items(), key=lambda item: item[1]))
    best_traits = list(b.items())
    worst_traits = list(b.items())[:5]
    return best_traits, worst_traits

5 tộc hệ tốt nhất và tệ nhất trong trò chơi này

In [35]:
a, b = get_best_worst_traits(list_traits)
print(a[-5:])

[('Set8_OxForce', 10648), ('Set8_Mascot', 11301), ('Set8_Brawler', 11791), ('Set8_StarGuardian', 12293), ('Set8_Aegis', 13821)]


In [36]:
print(b)

[('Set8_Arsenal', 2083), ('Set8_Corrupted', 2743), ('Set8_Forecaster', 3248), ('Set8_Deadeye', 3851), ('Set8_Recon', 4189)]


### VỊ TRÍ TRUNG BÌNH CỦA TỘC VÀ HỆ

In [37]:
list_traits_df = pd.read_csv('../Data/process_data_tft_general_csv.csv', usecols= ['Placement', 'List Traits','ID Match']).drop_duplicates().reset_index(drop=True)

Tộc và hệ tốt nhất và tệ nhất ở top 1. Ta làm tương tự như với món đồ.

In [38]:
traits_on_top = {}
for i in range(1,9,1):
    temp = convert_string_to_list(list_traits_df, i, 'List Traits')
    temp1 = 'top_' + str(i)
    traits_on_top[temp1] = get_best_worst_traits(temp)

5 tộc hệ tốt và tệ nhất trong top 1

In [42]:
best_traits_on_top_1, worst_traits_on_top_1 = traits_on_top['top_1']
print(best_traits_on_top_1[-5:])

[('Set8_Brawler', 1446), ('Set8_OxForce', 1450), ('Set8_Mascot', 1575), ('Set8_StarGuardian', 1626), ('Set8_Aegis', 1859)]


In [45]:
print(worst_traits_on_top_1)

[('Set8_Arsenal', 368), ('Set8_Recon', 379), ('Set8_Deadeye', 535), ('Set8_Corrupted', 541), ('Set8_Defender', 588)]


### TỈ LỆ CỦA TỘC HỆ ĐÓ TRONG TRẬN ĐẤU VÀ SỐ LẦN XUẤT HIỆN CỦA TỘC HỆ TRONG TỪNG TOP CỦA TRẬN ĐẤU

traits_avg_top: tộc hệ trung bình trong các top trận đấu

traits_avg_on_match: tần suất xuất hiện của tộc hệ trong trận đấu

In [46]:
all_traits_df = pd.read_csv('../Data/TocHe.csv', usecols=['Trait'])

In [47]:
all_traits = all_traits_df['Trait'].to_list()

**traits_avg_top**:  vị trí trung bình của tộc hệ khi ở trong top từ 1 đến 8

Nghĩa là tộc hệ đó nếu được kích hoạt thì vị trí trung bình của nó là bao nhiêu

**traits_avg_on_match**: tỉ lệ số trận đấu từ top 1 đến 8 mà tộc hệ đó được sử dụng

Nghĩa là mỗi top tần suất xuất hiện tộc hệ đó là bao nhiêu

In [42]:
sum = 0
temp = 0
traits_avg_top = []
traits_avg_on_match = []
for item in all_traits:
    temp1 = []
    temp2 = []
    for i in range(1,9,1):
        cc = dict(traits_on_top['top_' + str(i)][0])
        if(item in cc):
            temp = temp+dict(traits_on_top['top_' + str(i)][0])[item]*i
            sum = sum + (dict(traits_on_top['top_' + str(i)][0])[item]) 
        temp2.append(sum/32000)
        traits_avg_on_match.append(temp2)
    temp1.append(temp/sum)
    traits_avg_top.append(temp1)
    
    temp = 0
    sum = 0

nếu kích hoạt được tộc hệ đầu tiên này thì top của bạn sẽ là 4.31, tần suất xuât hiện của tộc hệ theo từng top cũng được hiển thị ra.

In [43]:
print(traits_avg_top[0], traits_avg_on_match[0])

[4.317058172426516] [0.03509375, 0.06921875, 0.103125, 0.13615625, 0.16725, 0.19740625, 0.2275625, 0.25409375]
